In [260]:
import pandas as pd
import os

# Overview

This project serves as a pre-processing step for other portfolio projects with the goal of using a subset of data from my business, Tryba Music LLC, in a public format while preserving financial and client privacy. Data is pulled from my Google Sheets time tracking and project management databases.


# Data Import

In [261]:
# Get data from Google Sheets
def google_sheets_to_dataframes(sheet_list):
    
    dfs = {}

    for sheet_name in sheet_list:
        
        # Define the URL for the API call
        url = "google_sheet_url" + sheet_name
        
        df = pd.read_csv(url)

        # Strip extra spaces in column names before writing .csv
        df.columns = df.columns.str.strip()
        
        # add dataframe to dictionary
        dfs[sheet_name] = df

    return dfs

# list of sheets to make dataframes
sheet_names = ['time_tracking', 'project_hours']

# get dataframes
dfs = google_sheets_to_dataframes(sheet_names)

# assigns the DataFrame (value) to a global variable with a name equal to the current sheet name
for key, value in dfs.items():
    globals()[key] = value

# Clarify names of dataframes for vs code interpreter formatting (makes it look better)   
time_tracking = time_tracking
project_hours = project_hours

print("Data loaded to dataframes.")

Data loaded to dataframes.


# Project Hours Cleanup

In [262]:
# Filter to only projects that are done and projects that are time tracked
project_hours = project_hours[(project_hours['time_tracked'] == True) & (project_hours['status'].str.lower() == "done")]

# filter to only desired columns
columns_to_drop = ['first_installment_paid', 'second_installment_paid', 'project_hourly', 'percent_budget_used',
                   'amount_paid', 'amount_pending', 'days_since_funded', 'days_to_start', 'days_to_finish', 
                   'days_between_payments', 'cataloged', 'licensing_agent', 'collab_agreement_signed',
                   'master_agreement_signed', 'pitched_to_licensing_agent', 'notes', 'disco.ac', 'time_tracked', 'status']

project_hours = project_hours.drop(columns=columns_to_drop)

In [263]:
# Cleanup price column. Changing format from object that is '$1,200.00' to a float. 

project_hours_to_float_dtype = ['price', 'total_additional_payments', 'expenses', 'total_after_fees']

for item in project_hours_to_float_dtype:
    project_hours[item] = project_hours[item].astype(str).str.replace('$', '').str.replace(',', '').str.replace('%', '').astype(float)

project_hours_to_datetime_dtype = ['fund_date', 'start_date', 'finish_date']

for item in project_hours_to_datetime_dtype:
    project_hours[item] = project_hours[item].astype('datetime64[ns]')

In [264]:
# Make 'songwriter' column boolean

project_hours['songwriter'].fillna('NaN')
project_hours['songwriter'] = project_hours['songwriter'].astype('bool')

In [265]:
# calculate project hourly
project_hours['project_hourly'] = project_hours.apply(lambda row: round(row['total_after_fees'] / 
                                                                        row['total_hours'], 2) if row['total_hours'] > 0 else None, axis=1)

# calculate days to finish projects
project_hours['days_to_finish'] = (project_hours['finish_date'] - project_hours['start_date']).dt.days

# calculate days to start projects
project_hours['days_to_start'] = (project_hours['start_date'] - project_hours['fund_date']).dt.days

In [266]:
# Add Client ID and Song ID Columns

# Create list of unique client_name
# Make client names strings
project_hours['client_string'] = project_hours['client'].apply(lambda x: str(x))

# Create list of unique client names
client_name = project_hours['client_string'].unique().tolist()

# Sort client_name alphabetically
client_name.sort()

# List of IDs for client_name
id_list = [x+1 for x in range(len(client_name))]

# Create dataframe of clients and ids
client_df = pd.DataFrame({'client_id':id_list, 'client_string':client_name})

# Filter original dataframe
song_df = project_hours[['client_string', 'project']]

# Inner merge to create new dataframe
client_project_df = pd.merge(song_df, client_df, on='client_string', how='inner')

# Add song_id by index
client_project_df['project_id'] = client_project_df.index + 1

# Sort dataframe
client_project_database = client_project_df.sort_values(by='client_id', ascending=True, ignore_index=True)

# Rename columns
client_project_database = client_project_database.rename(columns={'client_string':'client'})

# Merge dataframes
project_hours_with_ids = pd.merge(project_hours, client_project_df, how='left', on=["project", 'client_string'])

# Sort by client_id
project_hours_with_ids_sorted = project_hours_with_ids.sort_values(by='client_id', ignore_index=True, ascending=True)

# drop client_string column
project_hours_with_ids_sorted = project_hours_with_ids_sorted.drop(columns=['client_string'])


In [267]:
# Reorder Columns

cols_to_move = ['client', 'client_id', 'project', 'project_id']
remaining_cols = [col for col in project_hours_with_ids_sorted.columns if col not in cols_to_move]
new_col_order = cols_to_move + remaining_cols

# Reindex dataframe
project_hours_with_ids_sorted = project_hours_with_ids_sorted[new_col_order]

project_hours_with_ids_sorted

,client,client_id,project,project_id,fund_date,start_date,finish_date,price,total_additional_payments,expenses,...,experienced_client,difficult_client,backend_belief,backend_money_made,sync_license,fully_produced,songwriter,project_hourly,days_to_finish,days_to_start
0,AMiR,1,Victory,119,NaT,2022-01-06,2022-03-14,NaN,NaN,NaN,...,True,False,True,False,False,True,True,NaN,67.0,NaN
1,AMiR,1,Turnin It Up,120,NaT,2021-10-22,2021-11-03,NaN,NaN,NaN,...,True,False,True,False,False,True,True,NaN,12.0,NaN
2,Alexis Lynn,2,Our World To Choose,125,NaT,2020-09-02,2020-09-09,NaN,NaN,NaN,...,True,False,True,False,False,True,True,NaN,7.0,NaN
3,Ananya Kouds,3,Marble Eyes,37,2019-12-09,2020-02-03,2020-04-06,850.0,NaN,NaN,...,False,False,False,False,False,True,False,33.52,63.0,56.0
4,Angela Heart,4,Red,109,2022-11-30,2022-12-05,2023-01-17,2600.0,NaN,NaN,...,False,True,False,False,False,True,False,89.35,43.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,Trisha Shyam,63,Plaster,18,2024-01-31,2024-02-01,2024-04-23,2500.0,NaN,NaN,...,False,False,True,False,False,True,True,114.41,82.0,1.0
125,Trisha Shyam,63,MVP,17,2024-04-16,2024-04-22,2024-05-02,550.0,NaN,NaN,...,False,False,False,False,False,True,False,238.10,10.0,6.0
126,ZackSympsonMusic,64,It's Not The End,105,2021-11-24,2021-12-23,2022-01-09,1500.0,350.0,NaN,...,False,False,False,False,False,True,False,122.07,17.0,29.0
127,sabs,65,You Can't Run,22,NaT,2023-02-16,2023-03-08,NaN,NaN,NaN,...,True,False,True,False,False,True,True,NaN,20.0,NaN


In [268]:
# Create client and project keys for internal reference. 

client_key = client_df
project_key = client_project_df[['project_id', 'project']]

# Export keys as .csv files
client_key.to_csv('client_key.csv', index=False)
project_key.to_csv('project_key.csv', index=False)

In [269]:
project_hours_with_ids_sorted.columns

Index(['client', 'client_id', 'project', 'project_id', 'fund_date',
       'start_date', 'finish_date', 'price', 'total_additional_payments',
       'expenses', 'total_after_fees', 'total_hours', 'project_type',
       'master_ownership', 'song_ownership', 'soundbetter', 'spec_project',
       'experienced_client', 'difficult_client', 'backend_belief',
       'backend_money_made', 'sync_license', 'fully_produced', 'songwriter',
       'project_hourly', 'days_to_finish', 'days_to_start'],
      dtype='object')

In [270]:
# Get average project hourly gropued by client
paid_projects_finished = project_hours_with_ids_sorted[project_hours_with_ids_sorted['spec_project'] == False]

client_hourly = paid_projects_finished['project_hourly'].groupby(paid_projects_finished['client_id']).mean().to_frame(name='avg_hourly').reset_index()

# Determine max hourly
max_hourly = client_hourly['avg_hourly'].max()

# Calculate percent of max hourly
client_hourly['project_hourly_percent_of_max'] = client_hourly['avg_hourly'].apply(lambda x: x/max_hourly)

# Get total revenue grouped by client
client_revenue = paid_projects_finished['price'].groupby(paid_projects_finished['client_id']).sum().to_frame(name='revenue').reset_index()

# Determine max revenue
max_revenue = client_revenue['revenue'].max()

# Calculate percent of max revenue by client
client_revenue['client_revenue_percent_of_max'] = client_revenue['revenue'].apply(lambda x: x/max_revenue)

# Merge dataframes
client_hourly = client_hourly.merge(right=client_revenue, how='left', on='client_id')

# Percentage to weight revenue over hourly
weight_revenue = 0.6

# Calculate client score
client_hourly['score'] = (client_hourly['project_hourly_percent_of_max'] * (1 - weight_revenue) + client_hourly['client_revenue_percent_of_max'] * weight_revenue)*100

In [271]:
# Difficult clients: list of client_ids
difficult_clients = paid_projects_finished[paid_projects_finished['difficult_client'] == True]['client_id'].unique().tolist()

# Add column of difficult clients
client_hourly['difficult_client'] = client_hourly['client_id'].apply(lambda x: x in difficult_clients)


In [272]:
# Export .csv
client_hourly[['client_id', 'avg_hourly', 'revenue', 'score', 'difficult_client']].to_csv('client_stats.csv', index=False)

# Time Tracking Cleanup

In [273]:
# strip whitespace on ends of column names
time_tracking.columns = time_tracking.columns.str.strip()

# set datetime column
time_tracking['datetime'] = time_tracking['datetime'].astype('datetime64[ns]')

# Cleanup multi-day sessions

# back fill NaN values on sate "Started" 
'''
We want to copy all the 'Stopped' values to 'Started' to analyze values on 'Started' times 
to avoid sessions that straddle days
'''

time_tracking['hours'].fillna(method='bfill', inplace=True)


In [274]:
'''
Back fill notes column as well. They are currently lined up with stopped but since we switched to
analyzing on the start times we need to move the rows in the 'notes' column as well. 
'''

# Find indices of non-null values
value_indices = time_tracking[time_tracking['notes'].notna()].index

for idx in value_indices:
    if idx > 0: # start at the second row
        time_tracking.at[idx - 1, 'notes'] = time_tracking.at[idx, 'notes']

In [275]:
# Filter where state is started and stopped
time_tracking_started = time_tracking[time_tracking['state'].str.lower() == 'started']
time_tracking_stopped = time_tracking[time_tracking['state'].str.lower() == 'stopped']

# Reset indices
time_tracking_started = time_tracking_started.reset_index(drop=True)
time_tracking_stopped = time_tracking_stopped.reset_index(drop=True)

In [276]:
time_tracking_started.tail()

,activity,clock_in_out,state,hours,datetime,client,project,notes
9259,Client Production,"August 07, 2024 at 05:55PM",Started,0.42,2024-08-07 17:55:00,NaN,NaN,NaN
9260,Practicing,"August 08, 2024 at 09:10AM",Started,0.57,2024-08-08 09:10:00,NaN,NaN,NaN
9261,Spec,"August 08, 2024 at 09:45AM",Started,2.28,2024-08-08 09:45:00,NaN,NaN,NaN
9262,Edu Data Science,"August 09, 2024 at 07:41PM",Started,0.30,2024-08-09 19:41:00,NaN,NaN,NaN
9263,Spec,"August 10, 2024 at 12:55PM",Started,0.48,2024-08-10 12:55:00,NaN,NaN,NaN


In [277]:
time_tracking.tail()

,activity,clock_in_out,state,hours,datetime,client,project,notes
18523,Spec,"August 08, 2024 at 12:02PM",Stopped,2.28,2024-08-08 12:02:00,TRYBA,The Music's Not Loud Enough,NaN
18524,Edu Data Science,"August 09, 2024 at 07:41PM",Started,0.30,2024-08-09 19:41:00,NaN,NaN,NaN
18525,Edu Data Science,"August 09, 2024 at 07:59PM",Stopped,0.30,2024-08-09 19:59:00,NaN,NaN,NaN
18526,Spec,"August 10, 2024 at 12:55PM",Started,0.48,2024-08-10 12:55:00,NaN,NaN,NaN
18527,Spec,"August 10, 2024 at 01:24PM",Stopped,0.48,2024-08-10 13:24:00,Esabalu,Believe,NaN


In [278]:
time_tracking_stopped_to_merge = time_tracking_stopped[['client', 'project']]
time_tracking_started_to_merge = time_tracking_started.drop(columns=['client', 'project'])

merged_df_start_stop = time_tracking_started_to_merge.merge(time_tracking_stopped_to_merge,
                                                            left_index=True,
                                                            right_index=True,
                                                            how='left')

In [279]:
merged_df_start_stop.tail()

,activity,clock_in_out,state,hours,datetime,notes,client,project
9259,Client Production,"August 07, 2024 at 05:55PM",Started,0.42,2024-08-07 17:55:00,NaN,Ricky Asch,He's A Ghost
9260,Practicing,"August 08, 2024 at 09:10AM",Started,0.57,2024-08-08 09:10:00,NaN,NaN,NaN
9261,Spec,"August 08, 2024 at 09:45AM",Started,2.28,2024-08-08 09:45:00,NaN,TRYBA,The Music's Not Loud Enough
9262,Edu Data Science,"August 09, 2024 at 07:41PM",Started,0.30,2024-08-09 19:41:00,NaN,NaN,NaN
9263,Spec,"August 10, 2024 at 12:55PM",Started,0.48,2024-08-10 12:55:00,NaN,Esabalu,Believe


In [280]:
# Select desired columns
time_tracking = merged_df_start_stop[['activity','datetime', 'hours', 'client', 'project', 'notes']]

time_tracking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9264 entries, 0 to 9263
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   activity  9264 non-null   object        
 1   datetime  9264 non-null   datetime64[ns]
 2   hours     9264 non-null   float64       
 3   client    2851 non-null   object        
 4   project   2851 non-null   object        
 5   notes     223 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 434.4+ KB


In [281]:
# locate missing values between client and project
missing_value = time_tracking[pd.isnull(time_tracking['client']) & ~pd.isnull(time_tracking['project'])]
print(f'There are {len(missing_value)} missing values between client and project columns.')
missing_value

There are 0 missing values between client and project columns.


,activity,datetime,hours,client,project,notes


In [282]:
# Join time_tracking and client_song_database to encode client and project columns
time_tracking_merged = pd.merge(time_tracking, client_project_database, how='left', on=['client', 'project'])

In [283]:
time_tracking_merged

,activity,datetime,hours,client,project,notes,client_id,project_id
0,Client Production,2016-11-08 13:16:00,1.58,NaN,NaN,NaN,NaN,NaN
1,Client Production,2016-11-08 15:08:00,1.53,NaN,NaN,NaN,NaN,NaN
2,Client Production,2016-11-09 11:48:00,1.50,NaN,NaN,NaN,NaN,NaN
3,Client Production,2016-11-09 13:45:00,1.55,NaN,NaN,NaN,NaN,NaN
4,Client Production,2016-11-11 11:36:00,1.50,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
9259,Client Production,2024-08-07 17:55:00,0.42,Ricky Asch,He's A Ghost,NaN,49.0,3.0
9260,Practicing,2024-08-08 09:10:00,0.57,NaN,NaN,NaN,NaN,NaN
9261,Spec,2024-08-08 09:45:00,2.28,TRYBA,The Music's Not Loud Enough,NaN,NaN,NaN
9262,Edu Data Science,2024-08-09 19:41:00,0.30,NaN,NaN,NaN,NaN,NaN


# Scaling Financial Data

In [284]:
''' MinMix scaling that scales all monetary values across the entire dataframe. I chose to not use sci-kit scalers as they process per column. 
This is an abnormal case and I am using this method to obfuscate my financial data as a whole.
'''
columns_to_normalize = ['price', 'total_additional_payments', 'expenses', 'total_after_fees', 'project_hourly']
min_max_df = project_hours_with_ids_sorted[columns_to_normalize]

max_value = 0
min_value = 0 # setting to zero so that $0.00 is the minimum to keep scaling among all columns

# iterate through columns in dataframe to max value
for column in min_max_df.columns:
    max_column_value = min_max_df[column].max(skipna= True)
    if max_column_value > max_value:
        max_value = max_column_value

# define min max function
def min_max_global_scaler(x, min, max):
    x_scaled = (x - min)/(max - min)
    return x_scaled

# copy dataframe
project_hours_normalized = project_hours_with_ids_sorted.copy()

# normalize each column
columns_to_normalize = ['price', 'total_additional_payments', 'expenses', 'total_after_fees', 'project_hourly']
project_hours_normalized[columns_to_normalize] = project_hours_normalized[columns_to_normalize].apply(min_max_global_scaler, args=(min_value, max_value,))

In [285]:
# remove artist and song_name from project_hours for public viewing
project_hours_normalized = project_hours_normalized.drop(columns= ['client', 'project'])

# drop client and project columns from time_tracking
time_tracking_public = time_tracking_merged.copy().drop(columns= ['client', 'project'])


# Export CSV Files

In [ ]:
# Export .csv files for public analysis

# Define the target folder where you want to save the CSV files
target_folder = os.path.expanduser('~/Dropbox/01 Matt/Tryba Documents/GitHub/Portfolio-Projects')

# Define the filenames for your CSV files
time_tracking_filename = 'time-tracking.csv'
project_hours_normalized_filename = 'project-hours.csv'

# Create the full paths for saving the CSV files
time_tracking_path = os.path.join(target_folder + '/01 Time Tracking Personal Productivity', time_tracking_filename)
project_hours_normalized_path = os.path.join(target_folder + '/02 Music Business Client Analysis', project_hours_normalized_filename)

# Save the CSV files
time_tracking_public.to_csv(time_tracking_path, index=False)
project_hours_normalized.to_csv(project_hours_normalized_path, index=False)

In [ ]:
# Export .csv files for private analysis
time_tracking_merged.to_csv('time-tracking.csv', index=False)
project_hours_with_ids_sorted.to_csv('project-hours.csv', index=False)